<a href="https://colab.research.google.com/github/MathMachado/Python_RFB/blob/DS_Python/Notebooks/3DP_4_Data%20Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3DP_DATA TRANSFORMATION

# Leitura Recomendada:
* [Why, How and When to Scale your Features](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e)
* [Demonstrating the different strategies of KBinsDiscretizer](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html#sphx-glr-auto-examples-preprocessing-plot-discretization-strategies-py);
* [Why do we need feature scaling in Machine Learning and how to do it using SciKit Learn?](https://medium.com/@contactsunny/why-do-we-need-feature-scaling-in-machine-learning-and-how-to-do-it-using-scikit-learn-d8314206fe73)

# Machine Learning com Python (Scikit-Learn)

![Scikit-Learn](https://github.com/MathMachado/Python_RFB/blob/master/Material/scikit-learn-1.png?raw=true)

# Carregar as bibliotecas (genéricas) Python

In [0]:
!pip install category_encoders
!pip install update

In [0]:
import pandas as pd
from pandas import Series, DataFrame

import numpy as np
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')

import category_encoders as ce # library para aplicação do WOE - Weight Of Evidence para avaliar importância dos atributos

# remove warnings to keep notebook clean
import warnings
warnings.filterwarnings('ignore')

# Dados

## Dataframe gerado aleatoriamente - variáveis com distribuição Normal

In [0]:
N= 1000
df = pd.DataFrame({
    'Variavel1': np.random.normal(0, 2, N),
    'Variavel2': np.random.normal(5, 3, N),
    'Variavel3': np.random.normal(-5, 5, N),
    'Variavel4': np.random.normal(-10, 10, N)
})
df.head()

## Dataframe gerado aleatoriamente 2

In [0]:
from sklearn.datasets import make_classification

classification_data, classification_class = make_classification(n_samples=100, n_features=4, n_informative=3, n_redundant=1, n_classes=3)

df_classification = pd.DataFrame({'Feature 1':classification_data[:,0],
                                  'Feature 2':classification_data[:,1],
                                  'Feature 3':classification_data[:,2],
                                  'Feature 4':classification_data[:,3],
                                  'Class':classification_class})
df_classification.head()

# Transformações

## StandardScaler
* The StandardScaler assumes your data is normally distributed within each feature and will scale them such that the distribution is now centred around 0, with a standard deviation of 1.
* If data is not normally distributed, this is not the best scaler to use.

<img src="https://github.com/awantik/machine-learning-slides/blob/master/pp4.PNG?raw=true">

### Exemplo

In [0]:
df.head()

Histograma:

In [0]:
plt.hist(df['Variavel3'], color = 'blue', edgecolor = 'black', bins = int(180/5))

# Adiciona títulos e labels
plt.title('Histograma da Variável3')

Considere o gráfico a seguir:

In [0]:
df.plot.kde()

Qual a interpretação para esse gráfico?

A seguir, vamos aplicar a transformação StandardScaler as variáveis

In [0]:
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()

In [0]:
df_StandardScaler = standardscaler.fit_transform(df)
df_StandardScaler = pd.DataFrame(df_StandardScaler, columns=['Variavel1','Variavel2','Variavel3', 'Variavel4'])

Agora compare esse novo gráfico abaixo

In [0]:
df_StandardScaler.plot.kde()

Qual a conclusão?

## Normalize

In [0]:
df.plot.kde()

In [0]:
from sklearn.preprocessing import normalize

In [0]:
df_Normalize = normalize(df, norm= 'l2')
df_Normalize = pd.DataFrame(df_Normalize, columns=['Variavel1','Variavel2','Variavel3', 'Variavel4'])
df_Normalize.head()

In [0]:
df_Normalize.plot.kde()

## MinMaxScaler
* Transformação popular.
* Transforma os dados para o intervalo (0, 1);
* Se StandardScaler não é aplicável, então essa transformação funciona bem.
* Sensível aos outliers. Portanto, o ideal é que os outliers sejam tratados previamente.

<img src="https://github.com/awantik/machine-learning-slides/blob/master/pp3.PNG?raw=true">

### Exemplo

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
df.plot.kde()

In [0]:
minmax = MinMaxScaler()
df_MinMaxScaler = minmax.fit_transform(df)
df_MinMaxScaler = pd.DataFrame(df_MinMaxScaler,columns=['Variavel1','Variavel2','Variavel3', 'Variavel4'])
df_MinMaxScaler.plot.kde()

Qual a conclusão?

## RobustScaler
* Transformação ideal para dados com outliers;

<img src="https://github.com/awantik/machine-learning-slides/blob/master/pp2.PNG?raw=true">

In [0]:
df.plot.kde()

In [0]:
from sklearn.preprocessing import RobustScaler
robustscaler = RobustScaler()

In [0]:
df_RobustScaler = robustscaler.fit_transform(df)
df_RobustScaler = pd.DataFrame(df_RobustScaler, columns=['Variavel1','Variavel2','Variavel3', 'Variavel4'])
df_RobustScaler.plot.kde()

## Encoding Variáveis Categoricas

### Encoding Variáveis Ordinais
* Exemplo: Variáveis com valores ordinais: Low, Medium ou High.

In [0]:
# Gera um dataframe como exemplo.
# Aqui vou usar a função randint - Retorna números inteiros aleatórios incluindo o número inferior e excluindo o superior.

lIdade= [np.random.randint(20, 40), np.random.randint(20, 40), np.random.randint(20, 40), np.random.randint(20, 40), np.random.randint(20, 40),
         np.random.randint(20, 40), np.random.randint(20, 40),np.random.randint(20, 40),np.random.randint(20, 40),np.random.randint(20, 40)]

lSalario= ['Low', 'Medium', 'High']
lSalario2= np.random.choice(lSalario, 10, p=[0.6, 0.3, 0.1])

df = pd.DataFrame({
    'Idade': lIdade,
    'Salario': lSalario2})

In [0]:
df

Neste exemplo, vamos redefinir a variável categórical ordinal 'Salario' da seguinte forma:

In [0]:
df['Salario_Cat']= df.Salario.map({'Low':1,'Medium':2,'High':3})
df

### Encoding Variáveis Nominais
* Exemplo: Variáveis com valores nominais: Sexo (Feminino, masculino).

* Use One-Hot Encoding ou pd.get.dummies()

Vamos utilizar o dataframe criado no passo anterior:

In [0]:
df.Salario.unique()

In [0]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

#### Aplicar LabelEncoder()

In [0]:
le = LabelEncoder()
df['Salario_le'] = le.fit_transform(df.Salario)
df

#### Aplicar pd.get.dummies()

In [0]:
dummies= pd.get_dummies(df['Salario'])
df= pd.concat([df, dummies], axis= 1)
df

O comando a seguir produz o mesmo resultado que o anterior:
```
df= df.merge(dummies, how= 'left')
```

# Exercícios

## Exercício 1 - Predict Breast Cancer

In [0]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
X= cancer['data']
y= cancer['target']

df_cancer = pd.DataFrame(np.c_[X, y], columns= np.append(cancer['feature_names'], ['target']))
df_cancer['target'] = df_cancer['target'].map({0: 'malign', 1: 'benign'})
df_cancer.head()

## Exercício 2 - Predict Boston Housing Price

In [0]:
from sklearn.datasets import load_boston

boston = load_boston()
X= boston['data']
y= boston['target']

df_boston = pd.DataFrame(np.c_[X, y], columns= np.append(boston['feature_names'], ['target']))
df_boston.head()

## Exercício 3 - Predict Iris
* [Aqui](https://en.wikipedia.org/wiki/Iris_flower_data_set) você obterá mais informações sobre o dataframe iris. Confira.

In [0]:
from sklearn.datasets import load_iris

iris = load_iris()
X= iris['data']
y= iris['target']

df_iris = pd.DataFrame(np.c_[X, y], columns= np.append(iris['feature_names'], ['target']))
df_iris['target'] = df_iris['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
df_iris.head()

## Exercícios 4 - Diabetes

In [0]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X= diabetes['data']
y= diabetes['target']

df_diabetes = pd.DataFrame(np.c_[X, y], columns= np.append(diabetes['feature_names'], ['target']))
df_diabetes.head()

# WOE - Weight Of Evidence
* As vantagens da transformação WOE são
    * Lida bem com NaN's;
    * Lida bem com outliers;
    * A transformação é baseada no valor logarítmico das distribuições.
    * Usando a técnica de binning apropriada, pode estabelecer uma relação monotônica (aumentar ou diminuir) entre a variável dependente e independente.